# Medicina Inteligente: atravesando fronteras con Inteligencia Artificial

3 PREGUNTAS PARA LOS ALUMNOS


* ¿Qué es la optimización en el contexto del entrenamiento de modelos de machine learning?
* Define el concepto de overfitting en el contexto del machine learning. ¿Por qué es problemático y cómo se puede mitigar este fenómeno al entrenar modelos?

* ¿Cuáles son las principales diferencias entre el aprendizaje supervisado y el aprendizaje no supervisado en machine learning?




## ¿Qué es el Machine Learning y el Deep Learning?

El **aprendizaje automático** (*machine learning*) es un área de las ciencias computacionales que busca simular el proceso de aprendizaje humano mediante la exploración de datos y una optimización continua para mejorar el rendimiento de tareas específicas.

El **aprendizaje profundo** (*deep learning*) es una sub-área del aprendizaje automático que utiliza simula la propagación de información en el cerebro por medio de *redes neuronales*. Estas se organizan en capas, que se posicionan una después de otra, de forma que aumentan en *profundidad*.

## Algunos ejemplos de ML y DL en la vida diaria

* Alexa
* Google Maps
* Netflix
* Google Traductor
* Teclado del celular
* CAPTCHA

## Pasos para la implementación de algoritmos de ML y DL

1. Entender el problema
2. Adquisición y/o búsqueda de datos
3. Pre-procesamiento de datos
4. Extracción de características
5. Elección de modelo o arquitectura
6. Entrenamiento del modelo
7. Evaluación del modelo
8. Despliegue del modelo (en aplicaciones prácticas e industriales)

![imagen](/home/msr/Pictures/Screenshots/Screenshot\ from\ 2024-03-11\ 14-29-05.png)

## Métodos de Machine Learning

### K-nearest neighbors

---
Pensemos en un conjunto de datos como puntos en un  en un plano cartesiano, al entras un nuevo dato se mapea y se busca a los puntos mas cercanos para poderlo *clasificar* dentro de un grupo.  

>K = Al numero de vecinos que usa de referencia

>Distancia = Una formula para calcular la a distancia entre el dato de entrada y los demás datos. (Euclidiana)

>Selección = Elije los mas cercanos para despues ya sea calcular el promedio (problema de regresión) o solo la distancia (clasificación)

No hace predicciónes especificas sobre el funcionamiento de los datos, por eso normalmente se usa para, clasificaciones y reconocimiento de patrones y caracteristicas.
La elección de K es muy importante ya que un numero alto nos lleva a un **overfitting** y uno muy pequeño a un **underfitting**.

### Regresión lineal y logística
---
Mientras que la **regresión lineal**, el algoritmo es utilizado para predecir valores continuos encontrando la mejor linea de ajuste para los datos de salida (y = mx + b).

*   **Parámetros**: m y b (con valores aleatorios).
*   **Cálculo de la función de costo**: Para evaluar que tan bien se ajusta la linea a los datos (error cuadrático medio).



La **regresión logística** se usa para clasificaciones binarias (o sea dos conjuntos de datos) buscando la propabilidad de pertenencia a una clase.

*   Se utiliza la función sigmoide para el calculo de probabilidad.
*   Parámetros: m y b
*   **Función de costo**: Evalua la eficacia de la predicción (entropia cruzada).
*   Optimización de parametros: Minimiza la función de costo (descenso de gradiente)





### Principal Components Regression
---
Es un método de regresion lineal, utiliza un modelo estadistico: el **analisis de componentes principales**, que nos permite reducir la dimensionalidad de los datos seleccionando un numero de componentes principales.

Combina la capacidad predictiva de la **regresión lineal** con esta selección de datos para modelar la relación entre las variables independientes y la variable dependiente.

### Support Vector Machines
---
Son un tipo de algoritmo de **aprendizaje supervisado**, estos manejan conjuntos de datos de alta dimensionalidad, cararacterizado por ser un clasificador.


*   Busca encontrar el **hiperplano que maximice** el margen de discriminacion entre las clases de datos en un espacio de caracteristicas. (como verctor)
*   Utiliza la función **hinge loss** para penalizar las clasificaciones incorrectas.
*   Usa como **hiperparámetro** "C", que es el costo.
*   Si "C es muy grande elegirá un margen estricto, si es pequeño se optimizara el margen incluso si el hiperplano clasifica mal.



### Random Forests
---
Es otro algoritmo de aprendizaje de aprendizaje supervisado que combina multiples árboles de desición, cada uno entrenados por separado con diferentes datos. Basado en lógica binaria multivariante.


*   Utiliza **bagging** que es una técnica de ensamblaje, genera multiples conjuntos de datos de la base de datos original.
*   Para cada conjunto, se construye un árbol, con un subconjunto de **características** para cada uno.
###La Predicción
Cada árbol hace su predicción, dependiendo del algoritmo (si es de clasificación o regresión), la predicción final se dará por votación o promedio.






## Arquitecturas de Deep Learning

### MLP (Perceptrón Multicapa)

Es la arquitectura más simple de redes neuronales artificiales. Representa de manera sencilla los 3 tipos de capas que conforman a las redes neuronales artifuciales:

* **Capa de entrada**: conformada por las neuronas que representan a los valores de entrada ($x_i$)
* **Capa(s) oculta(s)**: conformada por las neuronas $h_i^j$ que reciben la suma de los productos de la capa anterior
* **Capa de salida**: la forma la(s) neurona(s) que representa(n) la predicción por la red neuronal. Es decir la capa al final de la arquitectura

#### Representación gráfica de un MLP

<img    src="images/MLP.png"
        width="300"
        height="300" />

#### Ejemplo de aplicación de un perceptrón multicapa

### CNN

### RNN

### Transformadores

## Métricas de evaluación de modelos

### Exactitud

Es la exactitud de la predicción del modelo, es una medida que nos informa sobre la correcta clasificación de todas las clasificaciónes. Su valor es en porcentajes de 0% a 100%.

### Sensibilidad

La especificidad es la verdadera tasa negativa de una prueba. En otras palabras, representa la proporción de personas sin la enfermedad, que tendrá un resultado negativo. Al igual que para la sensibilidad, los valores oscilan entre 0 y 100%.

### Especificidad

Es la tasa de positividad verdadera de una prueba. Es decir, cuántos sujetos con una enfermedad son realmente identificados como portadores de la enfermedad por la prueba. Los valores oscilan entre 0 y 100%.

### Precisión

Es una medida que nos dice de todos los casos previstos, cuántos son casos reales. Rango de valores posibles de 0 a 1.

### *Recall*
Es una medida informativa que indica el número de *casos* identificados por el modelo con respecto del número de *casos* real. Por ejemplo, una pregunta que el *recall* responde es: "Del número total de pacientes con cáncer, ¿cuántos fueron correctamente identificados por el modelo?".

### Curva ROC (*Area under the receiver operator curve*)

Es una medida que detalla qué tan sensible y específico es el modelo. Normalmente se detalla en una gráfica en donde el eje *x* representa las predicciones erróneas, y el *y* las positivas. Una métrica mala en términos de sensitividad y especificidad se encontraría por debajo de una valor de 0.5. Análogamente, sería como tirar un volado.

## Aprendizaje supervisado, no supervisado y reforzado

Son las dos principales clases en las que se clasifica el aprendizaje automático.

El **aprendizaje supervisado** consiste en un aprendizaje en donde el modelo aprende a mapear las entradas hacia una salida predefinida (etiquetas) con base en el ajuste continuo de sus parámetros. El **aprendizaje no supervisado** no cuenta con ningún tipo de identificador o etiqueta, sino que se encargan de encontrar estructuras o patrones inherentes a los datos de entrada.

### t-SNE

### PCA

# Comprendiendo el proceso


1.   Recopilación de datos

2.   Preparación de nuestros datos

3.   ¡Hora de elejir el modelo!

4.   Entrenar al modelo

5.   Evaluación: ¿Es suficientemente competente?

6.   Ajuste: Hora de volver a atrás

7.   Modelo listo para predecir













# Referencias
---

1. Auslander, N., Gussow, A. B., & Koonin, E. V. (2021). Incorporating machine learning into established bioinformatics frameworks. International journal of molecular sciences, 22(6), 2903.
2. Mendez, K. M., Reinke, S. N., & Broadhurst, D. I. (2019). A comparative evaluation of the generalised predictive ability of eight machine learning algorithms across ten clinical metabolomics data sets for binary classification. Metabolomics, 15, 1-15.
3. Rauschert, S., Raubenheimer, K., Melton, P. E., & Huang, R. C. (2020). Machine learning and clinical epigenetics: a review of challenges for diagnosis and classification. Clinical epigenetics, 12, 1-11.
